<a href="https://colab.research.google.com/github/cgaitan/03MAIR---Algoritmos-de-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos_Carlos_Ignacio_Gaitan_Mora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>

----
Nombre: Carlos Ignacio Gaitan Mora <br>
Link:   https://colab.research.google.com/drive/1WgV7wpEYJMf90Baby3R1DtdiQjGnm7Mw <br>
Github: https://github.com/cgaitan/03MAIR---Algoritmos-de-Optimizacion <br>


## Problema: Sesiones de doblaje
----

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día.

El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible. Los datos son:


Número de actores: 10

Número de tomas : 30

1 indica que el actor participa en la toma

0 en caso contrario


----



                                        

# Preparacion

In [ ]:
#Imports
import time
import random
import itertools
import math
import pandas as pd

In [ ]:
# Decorador para medir el tiempo de ejecución
def medir_tiempo(func):
    def wrapper(*args, **kwargs):
        inicio = time.perf_counter()
        resultado = func(*args, **kwargs)
        fin = time.perf_counter()
        print(f"Tiempo de ejecución de '{func.__name__}': {fin - inicio:.6f} segundos")
        return resultado
    return wrapper

In [ ]:
#Datos

# Generacion o lectura de los datos

def leer_datos_desde_excel(ruta_archivo):
    df = pd.read_excel(ruta_archivo, header=1)
    columnas_actores = df.iloc[:, 1:11]
    filas_tomas = columnas_actores.iloc[0:30]
    filas_tomas = filas_tomas.astype(int)
    matriz_participacion = filas_tomas.values.tolist()

    return matriz_participacion

def generar_datos_doblaje(total_actores, total_tomas):
    datos = []
    for toma in range(total_tomas):
        participacion = [random.randint(0, 1) for _ in range(total_actores)]
        datos.append(participacion)
    return datos

# Introducir la cantidad de actores y tomas
total_actores = 20
total_tomas = 17

# Generar y mostrar los datos
datos_doblaje = generar_datos_doblaje(total_actores, total_tomas)
datos_doblaje_excel = leer_datos_desde_excel('./datos_problema_doblaje.xlsx')
datos_doblaje_prueba = [
    [1, 0, 1, 0],
    [0, 1, 1, 0],
    [1, 0, 1, 0],
    [1, 0, 1, 0],
    [0, 0, 0, 1],
    [1, 0, 1, 0],
    [1, 1, 1, 0],
]

print('---- Datos del Excel')
for row in datos_doblaje_excel:
  print(row)

print('---- Datos generados aleatoriamente')
for row in datos_doblaje:
  print(row)

---- Datos del Excel
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 1, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 1, 0, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 1, 1, 0, 0]
[0, 1, 0, 1, 0, 0, 0, 1, 0, 0]
[1, 1, 0, 1, 1, 0, 0, 0, 0, 0]
[1, 1, 0, 1, 1, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 1, 0, 0, 0, 0]
[1, 1, 0, 1, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 1, 0, 0, 1, 0]
[1, 1, 1, 0, 1, 0, 0, 1, 0, 0]
[1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
[1, 0, 0, 1, 1, 0, 0, 0, 0, 0]
[1, 0, 1, 0, 0, 1, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
[1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 1, 0, 0, 0, 0]
[1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
[1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 1, 0, 0, 0, 0]
[1, 1, 0, 1, 0, 0, 0, 0, 0, 1]
[1, 0, 1, 0, 1, 0, 0, 0, 1, 0]
[0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 1, 1, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
---- Datos generados aleatoriamente
[1, 1, 0, 0, 

# Algoritmos


## Metodos comunes

In [ ]:
def calcular_costo(datos, tomas):
  submatriz = [datos[i-1] for i in tomas]
  return sum(1 for columna in zip(*submatriz) if any(columna))

## Fuerza bruta / Divide y venceras
Aquí tomamos todas las posibles combinaciones de tomas por día y las vamos dividiendo en los días siguientes, calculando las combinaciones posibles con las tomas restantes. De esta forma, analizamos todos los escenarios y obtenemos una respuesta para comparar con las siguientes soluciones.

Es una combinación de fuerza bruta, ya que generamos todas las combinaciones, pero las representamos y simplificamos utilizando el enfoque de "divide y vencerás".



In [ ]:
@medir_tiempo
def fuerza_bruta(datos, max_tomas_por_dia=6):
  num_tomas = len(datos)
  num_actores = len(datos[0])
  maximo_num_dias = math.ceil(num_tomas / max_tomas_por_dia)
  tomas = set(range(1, num_tomas + 1))
  print(f'Número de tomas: {num_tomas}, Número de actores: {num_actores}, Máximo número de días: {maximo_num_dias}')
  costo, asignacion = asignacion_fuerza_bruta(datos, tomas, max_tomas_por_dia, [])
  return costo, asignacion

def asignacion_fuerza_bruta(datos, tomas, max_tomas_por_dia, asignacion_actual):
  mejor_costo = math.inf
  mejor_asignacion = None

  # Casos base
  if len(tomas) == 0:
    return 0, asignacion_actual

  if len(tomas) <= max_tomas_por_dia:
    costo = calcular_costo(datos, tomas)
    return costo, asignacion_actual + [list(tomas)]

  for asignacion in itertools.combinations(tomas, max_tomas_por_dia):
    tomas_restantes = tomas.difference(asignacion)
    costo = calcular_costo(datos, asignacion)
    # Dividimos haciendo una recursion para calcular las siguientes restantes con la actual asignacion
    costo_validado, asignacion_validada = asignacion_fuerza_bruta(datos, tomas_restantes, max_tomas_por_dia, asignacion_actual + [list(asignacion)])
    costo_total = costo + costo_validado

    if costo_total < mejor_costo:
      mejor_costo = costo_total
      mejor_asignacion = asignacion_validada

  return mejor_costo, mejor_asignacion

print('------ Fuerza bruta/divide y venceras:')
print('---- DATOS PRUEBA')
costo, asignacion = fuerza_bruta(datos_doblaje_prueba)
print(f'-- Costo: {costo}')
print('Asignación de tomas por día:')
for i, dia in enumerate(asignacion):
  dia.sort()
  print(f'Día {i + 1}: Tomas {dia}')

print('---- DATOS GENERADOS')
costo, asignacion = fuerza_bruta(datos_doblaje)
print(f'-- Costo: {costo}')
print('Asignación de tomas por día:')
for i, dia in enumerate(asignacion):
  dia.sort()
  print(f'Día {i + 1}: Tomas {dia}')

print('---- DATOS EXCEL')
print('El notebook se va por tiempo...')


------ Fuerza bruta/divide y venceras:
---- DATOS PRUEBA
Número de tomas: 7, Número de actores: 4, Máximo número de días: 2
Tiempo de ejecución de 'fuerza_bruta': 0.000086 segundos
-- Costo: 4
Asignación de tomas por día:
Día 1: Tomas [1, 2, 3, 4, 6, 7]
Día 2: Tomas [5]
---- DATOS GENERADOS
Número de tomas: 17, Número de actores: 20, Máximo número de días: 3
Tiempo de ejecución de 'fuerza_bruta': 60.260723 segundos
-- Costo: 52
Asignación de tomas por día:
Día 1: Tomas [1, 2, 3, 7, 13, 16]
Día 2: Tomas [5, 8, 9, 10, 11, 14]
Día 3: Tomas [4, 6, 12, 15, 17]
---- DATOS EXCEL
El notebook se va por tiempo...


## Recocido simulado (Simulated annealing)
La idea aquí es usar un recocido simulado, ejecutándolo con 10 intentos y aumentando la temperatura en cada uno. Para la solución inicial, partiremos de la premisa de que la mejor estrategia es seleccionar las tomas desde la más costosa hasta la menos costosa y distribuirlas en los días. Luego, comenzaremos a tomar vecinos aleatorios para intercambiarlos y calcular nuevos costos, evaluando si obtenemos una mejor solución, hasta que el algoritmo se enfríe.

In [ ]:
import random


def recocido_simulado(datos, max_tomas_por_dia=6, temp_inicial=1000, solucion_inicial=None):
  if (solucion_inicial == None):
    asignacion_actual = crear_solucion(datos, max_tomas_por_dia)
  else:
    asignacion_actual = solucion_inicial

  costo_actual = sum(calcular_costo(datos, grupo) for grupo in asignacion_actual)

  mejor_asignacion = asignacion_actual[:]
  mejor_costo = costo_actual
  temperatura = temp_inicial
  enfriamiento = 0.99

  while temperatura > .0001:
    nueva_asignacion = genera_vecina_aleatorio(asignacion_actual, max_tomas_por_dia)
    nuevo_costo = sum(calcular_costo(datos, grupo) for grupo in nueva_asignacion)

    if nuevo_costo < mejor_costo:
      mejor_asignacion = nueva_asignacion[:]
      mejor_costo = nuevo_costo

    if nuevo_costo < costo_actual or probabilidad(temperatura, abs(costo_actual - nuevo_costo)):
      asignacion_actual = nueva_asignacion[:]
      costo_actual = nuevo_costo

    temperatura *= enfriamiento

  return mejor_costo, mejor_asignacion

def probabilidad(T, d):
  if random.random() <  math.exp(-1*d / T):
    return True
  else:
    return False

def crear_solucion(datos, max_tomas_por_dia):
  num_tomas = len(datos)
  tomas = list(range(1, num_tomas + 1))
  tomas.sort(key=lambda t: sum(datos[t-1]), reverse=True)
  asignacion = []
  while tomas:
    grupo = tomas[:max_tomas_por_dia]
    asignacion.append(grupo)
    tomas = tomas[max_tomas_por_dia:]
  return asignacion

def genera_vecina_aleatorio(asignacion, max_tomas_por_dia):
  vecino = [grupo[:] for grupo in asignacion]
  if len(vecino) < 2:
    return vecino

  i, j = random.sample(range(len(vecino)), 2)
  #print(f'dias escogidos a cambiar: {i+1} - {j+1}')
  if vecino[i] and vecino[j]:
    t1, t2 = random.choice(vecino[i]), random.choice(vecino[j])
    #print(f'tomas escogidas a cambiar: {t1} - {t2}')
    vecino[i].remove(t1)
    vecino[j].remove(t2)
    vecino[i].append(t2)
    vecino[j].append(t1)

  return [grupo for grupo in vecino if grupo]

@medir_tiempo
def seleccionar_mejor(rang_ini_temp, rang_fin_temp, intentos, data):
  mejor_costo = float('inf')
  mejor_asignacion = None
  for intento in range(0, intentos):
    temp = random.randint(rang_ini_temp, rang_fin_temp)
    # reusemos la mejor asignacion hasta el momento para calcular de nuevo.
    costo, asignacion = recocido_simulado(data, temp_inicial=temp, solucion_inicial=mejor_asignacion)
    #print(f'-- Intento {intento}, Temp: {temp}, Costo: {costo}')
    if (costo < mejor_costo):
      mejor_costo = costo
      mejor_asignacion = asignacion
  return mejor_costo, mejor_asignacion


print('------ Recocido simulado:')
print('---- DATOS PRUEBA')
costo, asignacion = recocido_simulado(datos_doblaje_prueba)
print(f'-- Costo: {costo}')
print('Asignación de tomas por día:')
for i, dia in enumerate(asignacion):
  dia.sort()
  print(f'Día {i + 1}: Tomas {dia}')

print('---- DATOS GENERADOS')
costo, asignacion = seleccionar_mejor(100000, 1000000, 10, datos_doblaje)
print(f'Mejor Asignación con costo {costo} de tomas por día:')
for i, dia in enumerate(asignacion):
  dia.sort()
  print(f'Día {i + 1}: Tomas {dia}')

print('---- DATOS EXCEL')
costo, asignacion = seleccionar_mejor(100000, 1000000, 10000, datos_doblaje_excel)
print(f'Mejor Asignación con costo {costo} de tomas por día:')
for i, dia in enumerate(asignacion):
  dia.sort()
  print(f'Día {i + 1}: Tomas {dia}')

------ Recocido simulado:
---- DATOS PRUEBA
-- Costo: 4
Asignación de tomas por día:
Día 1: Tomas [1, 2, 3, 4, 6, 7]
Día 2: Tomas [5]
---- DATOS GENERADOS
Tiempo de ejecución de 'seleccionar_mejor': 0.546901 segundos
Mejor Asignación con costo 52 de tomas por día:
Día 1: Tomas [1, 3, 8, 13, 16, 17]
Día 2: Tomas [5, 6, 9, 10, 11, 14]
Día 3: Tomas [2, 4, 7, 12, 15]
---- DATOS EXCEL
Tiempo de ejecución de 'seleccionar_mejor': 470.560826 segundos
Mejor Asignación con costo 27 de tomas por día:
Día 1: Tomas [2, 5, 11, 16, 20, 25]
Día 2: Tomas [1, 10, 12, 22, 26, 27]
Día 3: Tomas [14, 17, 18, 19, 23, 24]
Día 4: Tomas [3, 4, 8, 15, 21, 29]
Día 5: Tomas [6, 7, 9, 13, 28, 30]


#Modelo
El problema se identifica como un problema de combinatoria/asignación de recursos, ya que debemos optimizar las tomas por día para sacar el máximo provecho a los actores de voz que llamamos para ese día. La mejor estructura de datos para leer el problema y dar la solución es una matriz, donde en la entrada se identifiquen los actores necesarios por toma y en la salida se indiquen las tomas que deben grabarse por día, de modo que se sepa qué actores deben ser llamados cada día.

- ¿Como represento el espacio de soluciones?
 - Cada solución es una asignación de las 30 tomas de doblaje en grupos de hasta 6 tomas por día. Se representa como una lista de listas (una matriz), donde cada sublista contiene las tomas que se grabarán juntas en un mismo día. Del resultado, se tomó la que tuviese el mejor costo. Parte de la clave está en que, dado que se usa una heurística, en el caso de Excel se ejecuta N veces, variando la temperatura entre cada ejecución. En sí, el algoritmo no toma más de 1 segundo por ejecución, pero es necesario realizar varias ejecuciones para aproximarse lo mejor posible al resultado menos costoso.

- ¿Cual es la función objetivo?
 - El calculo del costo del día en general dado que es la que da el resultado importante para evaluar la solución:
    ```
    def calcular_costo(datos, tomas):
      submatriz = [datos[i-1] for i in tomas]
      return sum(1 for columna in zip(*submatriz) if any(columna))
    ```
 En el caso del recocido_simulado como tenemos que comparar todas las soluciones para encontrar la mejor sería la función que compara todos los costos dados por cada solucion encontrada para dar la mejor posible.

    ```
    def seleccionar_mejor(rang_ini_temp, rang_fin_temp, intentos, data):
      mejor_costo = float('inf')
      mejor_asignacion = None
      for intento in range(0, intentos):
        temp = random.randint(rang_ini_temp, rang_fin_temp)
        costo, asignacion = recocido_simulado(data, temp_inicial=temp)
        #print(f'-- Intento {intento}, Temp: {temp}, Costo: {costo}')
        if (costo < mejor_costo):
          mejor_costo = costo
          mejor_asignacion = asignacion
      return mejor_costo, mejor_asignacion
    ```

- ¿Como implemento las restricciones?
 - Al tener en cuenta el máximo de tomas por día, validar el costo y revisar los casos base, se tuvo en cuenta los escenarios planteados en el ejercicio, así como la precaución de no duplicar las tomas entre días.

#Diseño
Para el diseño, se pensó inicialmente en probar con algunos algoritmos básicos para obtener una solución que pudiera compararse con una heurística, dado que este es un problema tipo. Luego de encontrar un algoritmo que proporcionara una solución óptima analizando todas las posibles soluciones, se procedió a validar la heurística que mejor se aplicara a este caso.

- ¿Qué técnica utilizo? ¿Por qué?

 Inicialmente, Fuerza Bruta con Divide y Vencerás porque:
 - Sirvió para entender y analizar el problema.
 - Permitió obtener una solución para comparar intentos futuros.
 - Se quería ver cuánto tomaba evaluar todas las posibles soluciones y cómo se vería.

 Decidí utilizar la heurística de Recocido Simulado para la solución final dado que:
 - Es buena para problemas de combinatoria.
 - Bien implementada, proporciona diversificación e intensificación en la solución del problema.
 - El tiempo del algoritmo en sí es corto, dado que es muy eficiente, aunque es necesario evaluar la temperatura y ejecutarlo múltiples veces para obtener una buena solución.
 - Es simple de implementar para este problema.

# Análisis
- ¿Qué complejidad tiene el problema?
 - El problema es una combinatoria entre el total de tomas y las tomas que se pueden efectuar en el día, en el peor escenario puede crecer de forma exponencial o quizás factorial, teniendo n tomas y k tomas máximas por día:

 $$C(n, k) = \frac{n!}{k! \cdot (n-k)!}$$

- Orden de complejidad
 - Para fuerza bruta con divide y vencerás, teniendo n tomas y k tomas máximas por día sería en el peor caso (n igualmente de grande que m):

 $$O(C(n, k) * n * m)$$
 $$O(C(n, k) * n^2)$$

 - En el caso del recocido simulado, para cada intento, para n tomas, k tomas máximas y m número de actores sería en el peor caso (con k muy pequeño y n igualmente de grande que m):

  $$O(T_{max} * f_{costo})$$
  $$O(T_{max} * (n/k * n * m))$$
  $$O(T_{max} * n^3)$$

- Contabilizar el espacio de soluciones
 - En el caso del Excel sería C(30, 6) = 593775 soluciones posibles.

